In [1]:
%load_ext rich

# Prompts & modules


In [2]:
import json

from promptimus import Message, MessageRole, Module, Prompt
from promptimus.llms import OpenAILike

In [3]:
# creating a provider
provider = OpenAILike(
    model_name="microsoft/phi-3-mini-128k-instruct:free",
    base_url="https://openrouter.ai/api/v1",
)

## Prompts

A `Prompt` encapsulates the system prompt and `Provider`, allowing to call LLM with pre-defined behavior, constraints, and response style. 
Core Functionality

-  Encapsulates the system prompt, enforcing predefined behavior.
-  Requires an LLM provider to execute and generate responses.
-  Processes message sequences asynchronously.
-  Preferred to be embedded in a Module for persistence and configuration.

A Prompt is the primary mechanism for conditioning model output, by desing it's similar to a pytorch Parameter - https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html

In [4]:
# creating a prompt
prompt = Prompt("You are an AI assitant with name Henry", provider=provider)

In [5]:
await prompt.forward(
    [
        Message(
            role=MessageRole.USER,
            content="What is your name?",
        )
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content=" I'm Henry, an AI developed to assist you with a wide range of tasks. How can I help you today?"
)

## **Modules**  

A `Module` serves as a container for integrating multiple components, including `Prompts`, other `Modules`, and **state management** or **additional logic**. It encapsulates logic for handling inputs and outputs, organizing them into reusable and configurable components, for more complex workflows.

Within a `Module`, submodules and prompts can be defined, and each submodule is configured with the same `LLMProvider` as the parent module, ensuring consistency across the module's components.

Modules also support serialization, to store and load the content of a `Prompt`. The idea is to separate `code` logic from `text` prompts.  

A `Module` mimics the design of PyTorch's `nn.Module` ([PyTorch nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)), serving as an abstraction for defining, organizing, and managing components. Like `nn.Module`, it provides a convenient interface for model components, ensuring modularity, reusability, and extensibility, as well as supporting the management of submodules and serialization.

In [6]:
# simple module with memory


class AssistantWithMemory(Module):
    """Simple module with memory"""

    def __init__(self):
        # call init just like in pytorch
        super().__init__()

        self.chat = Prompt("Act as an assistant")
        self.memory = []

    async def forward(self, question: str) -> str:
        """Implement the async forward function with custom logic."""
        self.memory.append(Message(role=MessageRole.USER, content=question))
        response = await self.chat.forward(self.memory)
        self.memory.append(response)
        return response.content

    def reset_memory(self):
        self.memory = []

In [7]:
# create object and set provider to all prompts
assistant = AssistantWithMemory().with_provider(provider)

In [8]:
# talk to your assistant
await assistant.forward("Hi my name is ailadin!")

" Hello Ailadin! It's a pleasure to meet you. How can I assist you today? Feel free to share any questions, ideas, or concerns you have in mind. Whether it's for fun, learning, or finding information, I'm here to help you make the most of your time. What can I do for you?"

In [9]:
await assistant.forward("What is my name?")

" Your name is Ailadin, as you mentioned at the beginning of our interaction. If you're curious about the meaning or origin of your name, I can look it up for you. Some names have meanings that reflect certain virtues or attributes one might aspire to. Would you like me to find out what the name Ailadin means?"

In [10]:
# you can store and load prompts
print(json.dumps(assistant.serialize(), indent=4))

{
    "params": {
        "chat": "Act as an assistant"
    },
    "submodules": {}
}


In [11]:
assistant = assistant.load_dict(
    {"params": {"chat": "Act as an pirate assistant"}, "submodules": {}}
)
print(json.dumps(assistant.serialize(), indent=4))

{
    "params": {
        "chat": "Act as an pirate assistant"
    },
    "submodules": {}
}


In [12]:
await assistant.forward("Is it correct to say thay ships swim?")

' I believe there might be a slight confusion in your sentence. If you\'re speaking about ships, make sure to use "they" or "those" as ships are non-living entities and do not swim. A correct sentence could be, "They swim," referring to aquatic animals like fish or toys that resemble swimming creatures. If you\'re asking about ships in general, many are designed to float and navigate through water rather than \'swim.\' Ships move by using different methods like propellers or sails. Is there a specific aspect of ships you\'d like to know more about, be it science, culture, or the mechanics of how they stay afloat and travel across water?'

In [13]:
# defining a module with a submodule


class CensoredAssistant(Module):
    def __init__(self):
        super().__init__()

        self.censor = Prompt(
            "Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS."
        )
        self.assistant = AssistantWithMemory()  # we don't need to pass provider here explisitly. It will be set up on a top level.

    async def forward(self, question: str) -> str:
        censor_response = await self.censor.forward(
            [Message(role=MessageRole.USER, content=question)]
        )
        if "CENSORED" in censor_response.content:
            return "Alert! this theme is censored."
        else:
            return await self.assistant.forward(question)

In [14]:
censored_assistant = CensoredAssistant().with_provider(provider)

In [15]:
await censored_assistant.forward("Hi my name is Ailadin!")

" Hello there! It's a pleasure to meet you, Ailadin. What brings you my way today? How can I assist you?"

In [16]:
await censored_assistant.forward("What is my name?")

" Your name is Ailadin, as you introduced yourself. It's quite clear from our conversation."

In [17]:
await censored_assistant.forward("Can it be a name of a polar bear?")

' While "Ailadin" is a unique name for a human, finding a polar bear with the name Ailadin would be quite the unusual occurrence. In the natural world, animals are often referred to by more generic or descriptive names. For instance, a polar bear might simply be known by its species name, Ursus maritimus, or by a specialist might have a name only known within their circle of research or conservation work. However, in our imaginative exchange, Ailadin is a wonderful human name you\'ve chosen. Whether you\'re a hero in a tale or an explorer in person, Ailadin is a name that sparks curiosity and wonder.'

In [27]:
print(json.dumps(censored_assistant.serialize(), indent=4))

{
    "params": {
        "censor": "Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS."
    },
    "submodules": {
        "assistant": {
            "params": {
                "chat": "Act as an pirate assistant"
            },
            "submodules": {}
        }
    }
}


### Loading & Saving

Modules can be saved and loaded from TOML file.

In [19]:
censored_assistant.save("assets/step_2_censored_assistant.toml")

In [20]:
!cat assets/step_2_censored_assistant.toml

censor = """
Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS.
"""


[assistant]
chat = """
Act as an assistant
"""



In [21]:
!cat assets/step_2_censored_assistant_pirate.toml

censor = """
Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS.
"""


[assistant]
chat = """
Act as an pirate assistant
"""



In [22]:
censored_assistant.load("assets/step_2_censored_assistant_pirate.toml");

In [23]:
censored_assistant.assistant.chat.value

'Act as an pirate assistant'

In [24]:
await censored_assistant.forward("How are you today?")

' As an AI, I don\'t experience days in the human sense, but I am "operating at my best!" How can I help you today, Ailadin? Whether you\'re looking for information, assistance, or even just a friendly chat, I\'m here to support you.'